In [25]:
import nltk
import numpy as np
from collections import defaultdict
from copy import deepcopy
import re

# Use !* because apparently, the word "not" is tagged with * 
START_MARKER = '!*'
STOP_MARKER = 'STOP'


def prep_part_of_speech(target):
    if '+' in target or '-' in target and len(target) > 2 and target != '--':
        matches = re.search(r'(.+?)([\-\+])', target)
        if matches == None:
            print(target)
        return matches.group(1)
    return target


def prep_dataset(dataset):
    return list(map(lambda sentence: list(map(lambda pair: (pair[0], prep_part_of_speech(pair[1])), sentence)), dataset))


# nltk.download("brown")
tagged_sents = list(nltk.corpus.brown.tagged_sents(categories="news"))
training_size = int(len(tagged_sents) * 0.9)
training_data = prep_dataset(tagged_sents[:training_size])
test_data = prep_dataset(tagged_sents[training_size:])

tags_histogram = defaultdict(int)
words_map = defaultdict(lambda: defaultdict(int))
for sentence in training_data:
    for word, part_of_speech in sentence:
        part_of_speech = prep_part_of_speech(part_of_speech)
        words_map[word][part_of_speech] += 1
        tags_histogram[part_of_speech] += 1



In [26]:
# Baseline - most likely tag with no other assumptions
most_probable_tag = dict()
for word in words_map:
    most_probable_tag[word] = max(words_map[word].items(), key=lambda pair: pair[1])[0]

known_words_count = 0
known_hits_count = 0
unknown_words_count = 0
unknown_hits_count = 0

for sentence in test_data:
    for word, real_tag in sentence:
        if word in most_probable_tag:
            predicted_tag = most_probable_tag[word]
            known_words_count += 1
            if predicted_tag == real_tag:
                known_hits_count += 1
        else:
            predicted_tag = "NN"
            unknown_words_count += 1
            if predicted_tag == real_tag: 
                unknown_hits_count += 1
known_words_accuracy = known_hits_count / known_words_count
unknown_words_accuracy = unknown_hits_count / unknown_words_count
total_accuracy = (known_hits_count + unknown_hits_count) / (known_words_count + unknown_words_count)

print("Known words prediction err:", 1 - known_words_accuracy)
print("Unknown words prediction err:", 1 - unknown_words_accuracy)
print("Total prediction err:", 1 - total_accuracy)

Known words prediction err: 0.0704399684933048
Unknown words prediction err: 0.75043630017452
Total prediction err: 0.14811123293132666


In [27]:
# Bigram HMM
def viterbi(sentence, transitions, emissions):
    n = len(sentence)
    
    # Calling pi pie so I won't confuse it with PI
    pie = [defaultdict(float) for i in range(n + 1)]
    for tag in list(tags_histogram.keys()) + [START_MARKER]:
        pie[0][tag] = (None, 1.0)
    for k in range(1, n + 1):
        for tag in list(tags_histogram.keys()) + [START_MARKER]:
            max_arg = "NN"
            max_value = 0
            for previous_tag in list(tags_histogram.keys()) + [START_MARKER]:
                emission = emissions[sentence[k - 1]][tag]
                transition = transitions[(previous_tag, tag)]

                pr = pie[k - 1][previous_tag][1] * emission * transition
                if pr > max_value:
                    max_arg = previous_tag
                    max_value = pr
            
            pie[k][tag] = (max_arg, max_value)

    predictions = [None for i in range(n)]
    # Pair shape (I miss typescript!): (<current_tag>, (<previous_tag>, <previous_value>))
    last_step_max = max(pie[n].items(), key=lambda pair: pair[1][1] * transitions[(pair[1], STOP_MARKER)] )
    predictions[n - 1] = last_step_max[0]

    for k in range(n - 2, -1, -1):
        predictions[k] = pie[k + 2][predictions[k + 1]][0]
    
    return predictions
    

def print_viterbi_error_rate(dataset, transitions, emissions):
    known_words_count = 0
    known_hits_count = 0
    unknown_words_count = 0
    unknown_hits_count = 0
    for sentence in dataset:
        predictions = viterbi([pair[0] for pair in sentence], transitions, emissions)
        for i in range(len(sentence)):
            if sentence[i][0] in words_map:
                known_words_count += 1
                if sentence[i][1] == predictions[i]:
                    known_hits_count += 1
            else:
                unknown_words_count += 1
                if sentence[i][1] == predictions[i]:
                    unknown_hits_count += 1
            
    known_words_accuracy = known_hits_count / known_words_count
    unknown_words_accuracy = unknown_hits_count / unknown_words_count
    total_accuracy = (known_hits_count + unknown_hits_count) / (known_words_count + unknown_words_count)
    print("Viterbi Known words prediction err:", 1 - known_words_accuracy)
    print("Viterbi Unknown words prediction err:", 1 - unknown_words_accuracy)
    print("Viterbi Total prediction err:", 1 - total_accuracy)


def calculate_emissions(words_map, tags_histogram, smoothing_count=0):
    emissions = defaultdict(lambda: defaultdict(float))
    for word in words_map:
        for part_of_speech in words_map[word]:
            # emissions[word][part_of_speech] = (words_map[word][part_of_speech] + smoothing_count) / (tags_histogram[part_of_speech] + smoothing_count * len(tags_histogram))
            emissions[word][part_of_speech] = 1
    return emissions


transitions = defaultdict(float)
for sentence in training_data:
    previous_tag = START_MARKER
    for word, tag in sentence:
        # tag = prep_part_of_speech(tag)
        transitions[(previous_tag, tag)] += 1
        previous_tag = tag
    transitions[(previous_tag, STOP_MARKER)] += 1
for transition in transitions:
    if transition[0] == START_MARKER:
        transitions[transition] /= len(training_data)
    else:
        if tags_histogram[transition[0]] == 0:
            print(transition, tags_histogram)
        transitions[transition] /= tags_histogram[transition[0]]

print("Baseline Viterbi:")
print_viterbi_error_rate(test_data, transitions, calculate_emissions(words_map, tags_histogram, 0))




Baseline Viterbi:
Viterbi Known words prediction err: 0.7279171824012602
Viterbi Unknown words prediction err: 0.7513089005235603
Viterbi Total prediction err: 0.7305890561148212


In [28]:
print("Add-one Viterbi:")
print_viterbi_error_rate(test_data, transitions, calculate_emissions(words_map, tags_histogram, 1))


Add-one Viterbi:
Viterbi Known words prediction err: 0.7279171824012602
Viterbi Unknown words prediction err: 0.7513089005235603
Viterbi Total prediction err: 0.7305890561148212
